# Import

In [1]:
import os
import sys
module_path = os.path.join("./KeyPoint-Analysis/KPG/")
sys.path.insert(0, module_path)
from rouge_setbase import preprocess_dataset, compute_rouge, compute_rouge_max
from softF1 import softevaluation
root_dir_pth = "../"

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
2025-02-22 02:33:19.558607: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 02:33:19.584875: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-22 02:33:19.584900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-22 02:33:19.585588: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Una

In [2]:
sys.path.insert(1, '../')
from utils.prompting import *
from utils.postprocessing import *
from utils.g_eval import *

In [3]:
import pandas as pd
import re
import ast
import json
import statistics
import openai
from multiprocessing import Pool
openai.api_key = "YOUR API KEY"

# Read Predicted Summary

In [4]:
qqsum_output_path = "../output/atlas-xl-seed2-lgret-lglm"

In [5]:
ground_truth_df = pd.read_pickle("../data/test/test.pkl")
ground_truth_df = ground_truth_df.drop(columns=['reviews', 'reviewText', 'title', 'fact_check_article'])
ground_truth_df['retrieved_relevant_sent_len'] = ground_truth_df['retrieved_relevant_sent'].str.len()

predicted_df = pd.read_json(qqsum_output_path + "/test-result.jsonl", lines=True)
predicted_df['passages'] = predicted_df['passages'].apply(lambda x: [pas['text'] for pas in x])
predicted_df['passages_len'] = predicted_df['passages'].str.len()
predicted_df = predicted_df[['query', 'generation', 'passages', 'passages_scores', 'passages_len', 'comment_clusters', 'id']]
df = ground_truth_df.merge(predicted_df, on=['query', 'id'])

In [6]:
# df = df[df['passages'].str.len() > 0]
# df['passages_scores_min_thres_ref_ground'] = df.apply(lambda row: row['passages_scores'][:len(row['retrieved_relevant_sent'])][-1], axis=1)
# df['passages_scores_min_thres_ref_ground'].mean()

In [7]:
df['summary'] = df['generation'].apply(lambda x: re.findall("\[\/INST\] *((.+\n*)+)$", x)[0][0].replace("</s>", ""))
df['summary'] = df['summary'].apply(lambda x: re.sub("(Therefore|Thus)(.+\n*)+$", "", x))
df['final_summary'] = df['summary'].apply(lambda x: re.findall("(While[^\n]+\n+(\+ *[0-9]+[^\n]+\n*)+)", x))
df['final_summary'] = df['final_summary'].apply(lambda x: [e[0] for e in x])
df['final_summary_text'] = df['final_summary'].apply(lambda x: "\n\n".join(x[:1]))

## Post-process summary into KPs

### Trial

In [8]:
row = df.iloc[0]
print(row['id'])
get_kp_from_summary(row['final_summary_text'])

3609


'```json\n[{"key_point": "the old key components need to be taken to a locksmith or dealership to be reprogrammed and cut to fit the new key head", "prevalence": "7"}]\n```'

### Run

In [9]:
from utils.postprocessing import *

In [10]:
df['my_category'] = 1
num_workers = 1

In [11]:
inputs = [(qqsum_output_path + "/post_processed_cache",
           domain,
           df[df['my_category'] == domain].reset_index(drop=True)
           )
          for domain in df['my_category'].unique()]

In [12]:
start_time = time.time()
with Pool(num_workers) as processor:
    data = processor.starmap(prompted_claim_split_generation, inputs)
print("TIME ELAPSED", time.time() - start_time)

1 :  Loaded saved file. Done
TIME ELAPSED 0.12425708770751953


In [13]:
processed_df = pd.concat(data)
processed_df = processed_df[processed_df['comment_clusters'].str.len() > 0]
processed_df = processed_df[processed_df['final_summary'].str.len() > 0]

In [14]:
processed_df['claim_split_predicted'] = processed_df['claim_split_predicted'].apply(lambda x: re.sub(r"\n+ *", "", x.replace("json", "").replace("`", "")))
mask = processed_df['claim_split_predicted'].apply(lambda x: len(re.findall(r"(: *)\'((?:[^':]*\'+[^':,]*)+)\'( *)", x, re.DOTALL)) > 0)
processed_df.loc[mask, 'claim_split_predicted'] = processed_df.loc[mask, 'claim_split_predicted'].apply(
    lambda x: re.sub(r"(: *)\'((?:[^':]*\'+[^':]*)+)\'( *,)", r'\1"""\2"""\3', x))
processed_df['claim_split_predicted'] = processed_df['claim_split_predicted'].apply(extract_claims)

In [15]:
processed_df = processed_df.reset_index(drop=True)

In [16]:
processed_df = processed_df.apply(match_claim_with_cluster, axis=1)

In [17]:
processed_df = processed_df.rename(columns={'key_points': 'key_point_given'})

In [18]:
claim_split_predicted = pd.json_normalize(
    processed_df.to_dict(orient='records'), 
    "matching_comment_clusters", ["category", "asin", "id", "query", "passages", 'passages_len', 'key_point_given']
)
claim_split_predicted

,comments,cluster_size,key_point,prevalence,category,asin,id,query,passages,passages_len,key_point_given
0,[[the plastic case of my old keys was broken a...,7,the old key components need to be taken to a l...,7,Automotive,B006FDD74W,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,7,[Some Honda Accord models have a chip in the k...
1,"[In order to use this key, you will need two w...",17,you need two originally programmed keys to pro...,17,Automotive,B005OZNZD0,2799,I have a base model 2007 F150 XL V6. I would l...,"[In order to use this key, you will need two w...",22,"[If you have only one key, you can get the new..."
2,[I bought my 2003 Ford F250 truck from a used ...,5,it's possible to use a remake key or a key fro...,5,Automotive,B005OZNZD0,2799,I have a base model 2007 F150 XL V6. I would l...,"[In order to use this key, you will need two w...",22,"[If you have only one key, you can get the new..."
3,[It works best if you use a detailer as a lubr...,3,using a detailer as a lubricant,3,Automotive,B003POLA84,2501,"Do you need to use a lubricant? If so, what do...",[It works best if you use a detailer as a lubr...,3,[A blend of Windex and Dawn can be used as a l...
4,"[The cup-holders work great and adjust easily,...",16,the adjustable cup holders are easy to use and...,16,Automotive,B000FGEF6Y,3250,How sturdy are the adjustable cup holders? Wou...,"[The cup-holders work great and adjust easily,...",16,[Adjustable sliding tab for size customization...
...,...,...,...,...,...,...,...,...,...,...,...
109,[I will admit that I missed the slim look of t...,8,the 3DS is backwards compatible with DS games,8,Video_Games,B006QRMR4S,539,Can This replace my DS lite? Will I still be a...,[I will admit that I missed the slim look of t...,8,[Backwards compatible with original DS games]
110,[inFamous Second Son however doesn't keep you ...,16,Infamous Second Son is a bigger and more refin...,16,Video_Games,B00FYIXMHQ,1837,I've never played Infamous 1 - Should you find...,[inFamous Second Son however doesn't keep you ...,20,[Infamous 1 and 2 are fun and worth playing if...
111,"[In fact, before playing I was very skeptical ...",4,Infamous Second Son is a game that surpassed t...,4,Video_Games,B00FYIXMHQ,1837,I've never played Infamous 1 - Should you find...,[inFamous Second Son however doesn't keep you ...,20,[Infamous 1 and 2 are fun and worth playing if...
112,"[In terms of controls, the game allows you wit...",9,the game allows the use of Wii mote only,9,Video_Games,B00269DXXO,2189,do you have to use the remote and nunchuck? or...,"[In terms of controls, the game allows you wit...",9,[Game supports multiple controller options inc...


In [19]:
claim_split_predicted = post_process_kps(claim_split_predicted)

In [20]:
merged_df = claim_split_predicted.explode(['key_point_given'])
merged_df = merged_df.reset_index(drop=True)

# Evaluation

In [23]:
from softF1 import *

## G-Eval

### Trial

In [25]:
row = merged_df.iloc[0]
output_list = g_eval_relevancy(prompt, row['key_point_given'], row['key_point'])
statistics.mean(process_g_eval(output_list))

5

### Run

In [26]:
num_workers = 1
merged_df['my_category'] = 1

In [27]:
inputs = [(qqsum_output_path + "/g_eval_cache/gold_relevancy",
           domain,
           merged_df[merged_df['my_category'] == domain].reset_index(drop=True)
           )
          for domain in merged_df['my_category'].unique()]

In [28]:
start_time = time.time()
with Pool(num_workers) as processor:
    data = processor.starmap(prompted_g_eval_kp_relevancy, inputs)
print("TIME ELAPSED", time.time() - start_time)

1 :  Loaded saved file. Done
TIME ELAPSED 0.07189393043518066


In [29]:
g_eval_df = pd.concat(data)

In [30]:
g_eval_df['g_eval_scores'] = g_eval_df['g_eval_scores'].apply(process_g_eval)

In [31]:
g_eval_df['g_eval_relevancy'] = g_eval_df['g_eval_scores'].apply(lambda x: statistics.mean(x))

In [32]:
g_eval_df['g_eval_relevancy']

0     4.166667
1     4.687500
2     4.055556
3     4.105263
4     4.100000
        ...   
69    4.555556
70    4.263158
71    4.777778
72    3.350000
73    5.000000
Name: g_eval_relevancy, Length: 74, dtype: float64

### SoftPrecision

In [33]:
softp_data = g_eval_df.groupby(['category', 'asin', 'id', 'query', 'key_point'])['g_eval_relevancy'].max().reset_index()

In [34]:
P_average = softp_data['g_eval_relevancy'].mean()
P_average = (P_average - 1) / 4
P_average

0.8808026244123379

### SoftRecall

In [35]:
softr_data = g_eval_df.groupby(['category', 'asin', 'id', 'query', 'key_point_given'])['g_eval_relevancy'].max().reset_index()
softr_data

,category,asin,id,query,key_point_given,g_eval_relevancy
0,Automotive,B003POLA84,2501,"Do you need to use a lubricant? If so, what do...",A blend of Windex and Dawn can be used as a lu...,4.166667
1,Automotive,B003POLA84,2501,"Do you need to use a lubricant? If so, what do...",A lubricant is necessary when using clay.,4.687500
2,Automotive,B003POLA84,2501,"Do you need to use a lubricant? If so, what do...",Soap and water provide sufficient lubricity fo...,4.055556
3,Automotive,B003POLA84,2501,"Do you need to use a lubricant? If so, what do...",Water alone can also work as a lubricant.,4.105263
4,Beauty,B000VS8VZM,2816,What is the best shampoo to use with this prod...,Apply product for 30 minutes with a plastic cap,4.100000
...,...,...,...,...,...,...
59,Toys_and_Games,B002DWAVP2,2105,Is this puppet a good quality?.,Excellent quality for the price,4.555556
60,Toys_and_Games,B002DWAVP2,2105,Is this puppet a good quality?.,Loved by children,4.263158
61,Toys_and_Games,B002DWAVP2,2105,Is this puppet a good quality?.,Very good quality,4.777778
62,Video_Games,B00269DXXO,2189,do you have to use the remote and nunchuck? or...,Game supports multiple controller options incl...,3.350000


In [36]:
R_average = softr_data['g_eval_relevancy'].mean()
R_average = (R_average - 1) / 4
R_average

0.8215260645102338

### F1

In [37]:
result = softF1(P_average, R_average)

In [38]:
result

0.8501323138738053

## BARTScore

### SoftPrecision

In [39]:
softp_data = merged_df.groupby(['category', 'asin', 'id', 'query', 'key_point'])\
    .apply(lambda grp: "".join([cand + "=" for cand in (grp['key_point_given'].tolist())])).reset_index(name='multi_cands')
softp_data

,category,asin,id,query,key_point,multi_cands
0,Automotive,B003POLA84,2501,"Do you need to use a lubricant? If so, what do...",using a detailer as a lubricant,A blend of Windex and Dawn can be used as a lu...
1,Beauty,B000VS8VZM,2816,What is the best shampoo to use with this prod...,using a moisturizing conditioner after the pro...,Apply product for 30 minutes with a plastic ca...
2,Clothing_Shoes_and_Jewelry,B0044B5WN6,2578,Does anyone know why this product made my silv...,the product effectively removes tarnish from s...,Frequent cleaning may be required after using ...
3,Electronics,B0007M3PJO,551,I need a camcorder that will play old analog a...,this Sony digital 8 camcorder can play Digital...,Best to use compatible Sony 8mm video cassette...
4,Health_and_Personal_Care,B003C54M9A,2064,Is this good for getting rid of mice and bugs?,this product is effective in keeping mice away,100% peppermint oil prevents mice from enterin...
5,Home_and_Kitchen,B001DZ6V4A,682,How easy and fast is to clean the machine afte...,"cleaning the machine is quick, taking around 5...",Cleaning the pitcher section is straightforwar...
6,Home_and_Kitchen,B002X3FT8U,1359,Are these chairs comfortable enough to sit in ...,had issues with the chairs,Arm rests add to comfort=Comfortable for sitti...
7,Home_and_Kitchen,B002X3FT8U,1359,Are these chairs comfortable enough to sit in ...,the chairs are comfortable and sturdy,Arm rests add to comfort=Comfortable for sitti...
8,Office_Products,B00434C7R8,2140,how to have the printer recognize the remanufa...,the printer failed to recognize the new cartridge,Buying remanufactured cartridges can lead to i...
9,Office_Products,B004G5O2E0,211,How comfortable is this chair? I want one that...,the chair provides good back support,Easy to assemble=Firm yet cushioned feel=Great...


In [40]:
cands, refs= preprocess_text(softp_data, metrics = "softPrecision")

#BART score cannot be processed for different numbers of reference sentences, so
#check for the maximum number of reference sentences and match the size.
#We fill in the None for the missing sentences because we only pick one of the 
#maximum values and not the average, so there is no impact on performance

refs = balance_ref_num(refs)        

# generation scores from the first list of texts to the second list of texts.
P = bart_scorer.multi_ref_score(cands, refs, agg="max", batch_size=4) # agg means aggregation, can be mean or max

#mapping the score to (0,1]
P_average = math.tanh(math.exp((mean(P))/2+1.3))
#P_average = math.tanh(mean(P)) + 1
#P_average = math.exp(mean(P))

In [41]:
P_average

0.48380538815792623

### SoftRecall

In [42]:
softr_data = merged_df.groupby(['category', 'asin', 'id', 'query', 'key_point_given'])\
    .apply(lambda grp: "".join([cand + "=" for cand in (grp['key_point'].tolist())])).reset_index(name='multi_cands')
softr_data

,category,asin,id,query,key_point_given,multi_cands
0,Automotive,B003POLA84,2501,"Do you need to use a lubricant? If so, what do...",A blend of Windex and Dawn can be used as a lu...,using a detailer as a lubricant=
1,Automotive,B003POLA84,2501,"Do you need to use a lubricant? If so, what do...",A lubricant is necessary when using clay.,using a detailer as a lubricant=
2,Automotive,B003POLA84,2501,"Do you need to use a lubricant? If so, what do...",Soap and water provide sufficient lubricity fo...,using a detailer as a lubricant=
3,Automotive,B003POLA84,2501,"Do you need to use a lubricant? If so, what do...",Water alone can also work as a lubricant.,using a detailer as a lubricant=
4,Beauty,B000VS8VZM,2816,What is the best shampoo to use with this prod...,Apply product for 30 minutes with a plastic cap,using a moisturizing conditioner after the pro...
...,...,...,...,...,...,...
59,Toys_and_Games,B002DWAVP2,2105,Is this puppet a good quality?.,Excellent quality for the price,the puppet is of high-quality and durable=
60,Toys_and_Games,B002DWAVP2,2105,Is this puppet a good quality?.,Loved by children,the puppet is of high-quality and durable=
61,Toys_and_Games,B002DWAVP2,2105,Is this puppet a good quality?.,Very good quality,the puppet is of high-quality and durable=
62,Video_Games,B00269DXXO,2189,do you have to use the remote and nunchuck? or...,Game supports multiple controller options incl...,the game allows the use of Wii mote only=


In [43]:
cands, refs= preprocess_text(softr_data, metrics = "softRecall")

refs = balance_ref_num(refs)

R = bart_scorer.multi_ref_score(cands, refs, agg="max", batch_size=4)

#mapping the score to (0,1]
R_average = math.tanh(math.exp((mean(R)/2)+1.3)) 
#R_average = (0.25 * mean(R)) + 1
#R_average = math.exp(mean(R))

In [44]:
R_average

0.2658728659404454

### F1

In [45]:
result = softF1(P_average, R_average)

In [46]:
result

0.34316248178141434

## BLEURTScore

In [47]:
softp_data = merged_df.sort_values(by=['category', 'asin', 'id', 'query', 'key_point'])
df_compare_precision = softp_data[['key_point', 'key_point_given']].rename(columns={'key_point': 'candidate', 'key_point_given': 'reference'})
df_compare_precision

,candidate,reference
0,using a detailer as a lubricant,A blend of Windex and Dawn can be used as a lu...
1,using a detailer as a lubricant,A lubricant is necessary when using clay.
2,using a detailer as a lubricant,Soap and water provide sufficient lubricity fo...
3,using a detailer as a lubricant,Water alone can also work as a lubricant.
4,using a moisturizing conditioner after the pro...,Apply product for 30 minutes with a plastic cap
...,...,...
69,the puppet is of high-quality and durable,Excellent quality for the price
70,the puppet is of high-quality and durable,Loved by children
71,the puppet is of high-quality and durable,Very good quality
72,the game allows the use of Wii mote only,Game supports multiple controller options incl...


In [48]:
candidates = df_compare_precision['candidate']
references = df_compare_precision['reference']

In [49]:
import tensorflow as tf

In [50]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2025-02-22 02:34:00.806007: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:00.838783: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:00.839009: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [51]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8995474159845660884
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 19566428160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16942633511190395554
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2025-02-22 02:34:01.143264: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:01.143660: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:01.143985: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:01.144511: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:01.144786: I external/local_xla/xla/stream_executor

### SoftPrecision

In [52]:
softp_data = merged_df.sort_values(by=['category', 'asin', 'id', 'query', 'key_point'])
df_compare_precision = softp_data[['key_point', 'key_point_given']].rename(columns={'key_point': 'candidate', 'key_point_given': 'reference'})
df_compare_precision

,candidate,reference
0,using a detailer as a lubricant,A blend of Windex and Dawn can be used as a lu...
1,using a detailer as a lubricant,A lubricant is necessary when using clay.
2,using a detailer as a lubricant,Soap and water provide sufficient lubricity fo...
3,using a detailer as a lubricant,Water alone can also work as a lubricant.
4,using a moisturizing conditioner after the pro...,Apply product for 30 minutes with a plastic cap
...,...,...
69,the puppet is of high-quality and durable,Excellent quality for the price
70,the puppet is of high-quality and durable,Loved by children
71,the puppet is of high-quality and durable,Very good quality
72,the game allows the use of Wii mote only,Game supports multiple controller options incl...


In [53]:
candidates = df_compare_precision['candidate']
references = df_compare_precision['reference']

In [54]:
import tensorflow as tf

In [55]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [56]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9377821508395099655
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 19566428160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16850509119386918036
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2025-02-22 02:34:03.232231: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:03.232829: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:03.233332: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:03.234090: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:03.234103: I tensorflow/core/common_runtime/gpu/gpu

In [57]:
result = calculatingScore(references, candidates)
df_compare_precision["BLEURT Score"] = result

#After calculating the semantic quality of all candidates and reference pairs, the one with the highest score is selected as the correct pair.
df_bestkp_pair_precision = df_compare_precision.loc[df_compare_precision.groupby(["candidate"])["BLEURT Score"].idxmax()]
#take average of all best scores as the soft precision score.
P_average = df_bestkp_pair_precision["BLEURT Score"].mean()

INFO:tensorflow:Reading checkpoint /mnt/d/Desktop/PHD READING/QQSUMM/evaluation/KeyPoint-Analysis/KPG/BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: /mnt/d/Desktop/PHD READING/QQSUMM/evaluation/KeyPoint-Analysis/KPG/BLEURT-20/sent_piece.model.
INFO:tensorflow:SentencePiece tokenizer created.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


2025-02-22 02:34:05.713791: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:05.714179: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:05.714448: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:05.714851: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:05.714862: I tensorflow/core/common_runtime/gpu/gpu

INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [58]:
P_average

0.5045840508408017

### SoftRecall

In [59]:
softr_data = merged_df.sort_values(by=['category', 'asin', 'id', 'query', 'key_point_given'])
df_compare_recall = softp_data[['key_point', 'key_point_given']].rename(columns={'key_point_given': 'candidate', 'key_point': 'reference'})
df_compare_recall

,reference,candidate
0,using a detailer as a lubricant,A blend of Windex and Dawn can be used as a lu...
1,using a detailer as a lubricant,A lubricant is necessary when using clay.
2,using a detailer as a lubricant,Soap and water provide sufficient lubricity fo...
3,using a detailer as a lubricant,Water alone can also work as a lubricant.
4,using a moisturizing conditioner after the pro...,Apply product for 30 minutes with a plastic cap
...,...,...
69,the puppet is of high-quality and durable,Excellent quality for the price
70,the puppet is of high-quality and durable,Loved by children
71,the puppet is of high-quality and durable,Very good quality
72,the game allows the use of Wii mote only,Game supports multiple controller options incl...


In [60]:
candidates = df_compare_recall['candidate']
references = df_compare_recall['reference']

In [61]:
import tensorflow as tf

In [62]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [63]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11561897595893079543
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 19566428160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2268529874114870919
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2025-02-22 02:34:22.376180: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:22.376605: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:22.376769: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:22.377159: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 02:34:22.377171: I tensorflow/core/common_runtime/gpu/gpu

In [64]:
result = calculatingScore(references, candidates)
df_compare_recall["BLEURT Score"] = result

#After calculating the semantic quality of all candidates and reference pairs, the one with the highest score is selected as the correct pair. 
df_bestkp_pair_recall = df_compare_recall.loc[df_compare_recall.groupby(["candidate"])["BLEURT Score"].idxmax()]

#take average of all best scores as the soft precision score.
R_average = df_bestkp_pair_recall["BLEURT Score"].mean()

INFO:tensorflow:Reading checkpoint /mnt/d/Desktop/PHD READING/QQSUMM/evaluation/KeyPoint-Analysis/KPG/BLEURT-20.


INFO:tensorflow:Reading checkpoint /mnt/d/Desktop/PHD READING/QQSUMM/evaluation/KeyPoint-Analysis/KPG/BLEURT-20.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Will load checkpoint BLEURT-20


...

INFO:tensorflow:Creating Eager Mode predictor.


INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [65]:
R_average

0.4138677236624062

### F1

In [66]:
result = softF1(P_average, R_average)

In [67]:
result

0.45474581968290173

## ROUGE

In [68]:
gt_gold_kp = merged_df

In [69]:
predictions, references = [], []
for topic in sorted(gt_gold_kp['id'].unique()):
    kps = gt_gold_kp.loc[(gt_gold_kp['id']==topic), 'key_point'].unique().tolist()
    gold_kps = gt_gold_kp.loc[(gt_gold_kp['id']==topic), 'key_point_given'].unique().tolist()
    if len(kps) > 0 and len(gold_kps) > 0:
        predictions.append(kps)
        references.append(gold_kps)

In [70]:
compute_rouge(predictions, references)

Rouge 1: 0.256
Rouge 2: 0.061
Rouge L: 0.22
